##### RM 335817 – Felipe Filgueira
##### RM 336094 – Jefferson Kwak
##### RM 336524 – Rodolfo Moreira

## Classificador de sentimentos com o dataset imdb-reviews-pt-br.csv



In [ ]:
import pandas as pd
import numpy as np

In [3]:
#Importando o dataset imdb-reviews-pt-br.csv

#Importando pelo link
dados = pd.read_csv("https://dados-ml-pln.s3-sa-east-1.amazonaws.com/imdb-reviews-pt-br.csv")

#Importante pela base local
#dados = pd.read_csv("dados/imdb-reviews-pt-br.csv", index_col=0)

dados.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [4]:
#Analisando o dataset
print(dados.shape)
dados.describe()

(49459, 4)


,id
count,49459.000000
mean,24730.960917
std,14277.792868
min,1.000000
25%,12366.500000
50%,24731.000000
75%,37095.500000
max,49460.000000


In [5]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49459 entries, 0 to 49458
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         49459 non-null  int64 
 1   text_en    49459 non-null  object
 2   text_pt    49459 non-null  object
 3   sentiment  49459 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.5+ MB


In [6]:
#Verificando valores nulos
dados.isnull().sum()

id           0
text_en      0
text_pt      0
sentiment    0
dtype: int64

In [ ]:
#**Provisoria** DataSet foi diminuido para efeitos de testes mais rapidos
#dados = dados.sample(1000)
#dados.shape

In [8]:
#Verificando a quantidade e porcetagem das classificações

quantidade_pos_neg = dados.sentiment.value_counts()
porcentagem_pos_neg = dados.sentiment.value_counts(normalize=True) * 100
df_pos_neg = pd.DataFrame(data = porcentagem_pos_neg)
df_pos_neg.insert(1, value = quantidade_pos_neg, column = 'quantidade')
df_pos_neg

,sentiment,quantidade
neg,50.071777,24765
pos,49.928223,24694


In [9]:
# Instalar as bibliotecas spacy
!pip install spacy
!python -m spacy download pt

     |████████████████████████████████| 21.2MB 1.2MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-cp36-none-any.whl size=21186282 sha256=6e72fbb155b6cbb476da2aae7d83638b9300fc7b535216935c0d44fa457a084b
  Stored in directory: /tmp/pip-ephem-wheel-cache-azn295ct/wheels/ea/94/74/ec9be8418e9231b471be5dc7e1b45dd670019a376a6b5bc1c0
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')


In [10]:
# Importando as bibliotecas e as stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import nltk
import spacy
import string
import time

# stopwords NLTK
nltk.download('stopwords')
stops_nltk = nltk.corpus.stopwords.words('portuguese')

# stopwords SpaCy
nlp = spacy.load('pt')
stops_spacy = nlp.Defaults.stop_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#Tentativa 1 (Modelo DecisionTreeClassifier + Unigrama)

start = time.time()

vect = CountVectorizer(ngram_range=(1,1))
vect.fit(dados.text_pt)
text_vect = vect.transform(dados.text_pt)

X_train, X_test, y_train, y_test = train_test_split(
      text_vect, 
      dados["sentiment"], 
      test_size = 0.2, 
      random_state = 42
  )

modelo = DecisionTreeClassifier(random_state=42)
modelo.fit(X_train, y_train)

y_prediction = modelo.predict(X_test)

f1 = f1_score(y_prediction, y_test, average = 'weighted')

print("A métrica F1 Score foi de:", f1)

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Tempo de processamento: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

## A métrica F1 Score foi de: 0.7053953960481304 com um tempo de processamento de 00:01:53.05

A métrica F1 Score foi de: 0.7053953960481304
Tempo de processamento: 00:02:03.80


In [ ]:
#Tentativa 2 (Modelo DecisionTreeClassifier + Unigrama retirando stopwords nltk)
start = time.time()

vect = CountVectorizer(ngram_range=(1,1), stop_words=stops_nltk)
vect.fit(dados.text_pt)
text_vect = vect.transform(dados.text_pt)

X_train, X_test, y_train, y_test = train_test_split(
      text_vect, 
      dados["sentiment"], 
      test_size = 0.2, 
      random_state = 42
  )

modelo = DecisionTreeClassifier(random_state=42)
modelo.fit(X_train, y_train)

y_prediction = modelo.predict(X_test)

f1 = f1_score(y_prediction, y_test, average = 'weighted')
print("A métrica F1 Score foi de:", f1)

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Tempo de processamento: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

## A métrica F1 Score foi de: 0.7226846988737454 com um tempo de processamento de 00:01:44.06
# Retirando as stopwords NTK e tivemos uma melhora.

A métrica F1 Score foi de: 0.7226846988737454
Tempo de processamento: 00:01:51.08


In [ ]:
#Tentativa 3 (Modelo DecisionTreeClassifier + Unigrama retirando stopwords nltk e suas pontuacoes)
start = time.time()

vect = CountVectorizer(ngram_range=(1,1), stop_words=stops_nltk + list(string.punctuation))
vect.fit(dados.text_pt)
text_vect = vect.transform(dados.text_pt)

X_train, X_test, y_train, y_test = train_test_split(
      text_vect, 
      dados["sentiment"], 
      test_size = 0.2, 
      random_state = 42
  )

modelo = DecisionTreeClassifier(random_state=42)
modelo.fit(X_train, y_train)

y_prediction = modelo.predict(X_test)

f1 = f1_score(y_prediction, y_test, average = 'weighted')
print("A métrica F1 Score foi de:", f1)

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Tempo de processamento: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

## A métrica F1 Score foi de: 0.7226846988737454 com um tempo de processamento de 00:01:44.11
## Não tivemos melhoras ao retirar as pontuaçoes

A métrica F1 Score foi de: 0.7226846988737454
Tempo de processamento: 00:01:51.78


In [ ]:
#Tentativa 4 (Modelo DecisionTreeClassifier + Unigrama sem stopwords spacy)
start = time.time()

vect = CountVectorizer(ngram_range=(1,1), stop_words=stops_spacy)
vect.fit(dados.text_pt)
text_vect = vect.transform(dados.text_pt)

X_train, X_test, y_train, y_test = train_test_split(
      text_vect, 
      dados["sentiment"], 
      test_size = 0.2, 
      random_state = 42
  )

modelo = DecisionTreeClassifier(random_state=42)
modelo.fit(X_train, y_train)

y_prediction = modelo.predict(X_test)

f1 = f1_score(y_prediction, y_test, average = 'weighted')
print("A métrica F1 Score foi de:", f1)

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Tempo de processamento: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

## A métrica F1 Score foi de: 0.7242176237395925 com um tempo de processamento de 00:01:39.15
# Tivemos uma pequena melhora utilizando os stopwords spacy

A métrica F1 Score foi de: 0.7242176237395925
Tempo de processamento: 00:01:45.15


In [ ]:
#Tentativa 5 (Modelo RandomForestClassifier + Unigrama sem stopwords spacy)
start = time.time()

#Instanciando o modelo
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops_spacy)
vect.fit(dados.text_pt)
text_vect = vect.transform(dados.text_pt)

X_train, X_test, y_train, y_test = train_test_split(
      text_vect, 
      dados["sentiment"], 
      test_size = 0.2, 
      random_state = 42
  )

modelo = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state = 42)

modelo.fit(X_train, y_train)

y_prediction = modelo.predict(X_test)

f1 = f1_score(y_prediction, y_test, average = 'weighted')
print("A métrica F1 Score foi de:", f1) 

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Tempo de processamento: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds)) 
## A métrica F1 Score foi de: 0.8561864843167125 com um tempo de processamento de 00:03:59.46
# Tivemos uma grande melhora ao utilizar o modelo RandomForestClassifier

A métrica F1 Score foi de: 0.8561864843167125
Tempo de processamento: 00:03:22.41


In [ ]:
#Tentativa 6 (Modelo RandomForestClassifier + Unigrama sem stopwords spacy + n_estimators 200)
start = time.time()

#Instanciando o modelo
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops_spacy)
vect.fit(dados.text_pt)
text_vect = vect.transform(dados.text_pt)

X_train, X_test, y_train, y_test = train_test_split(
      text_vect, 
      dados["sentiment"], 
      test_size = 0.2, 
      random_state = 42
  )

modelo = RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state = 42)

modelo.fit(X_train, y_train)

y_prediction = modelo.predict(X_test)

f1 = f1_score(y_prediction, y_test, average = 'weighted')
print("A métrica F1 Score foi de:", f1) 

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Tempo de processamento: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds)) 
## A métrica F1 Score foi de: 0.8591866574453395 com um tempo de processamento de 00:07:49.91
# Aumentamos o parametro de n_estimators e tivemos uma pequena melhora, mas o tempo de processamento foi o dobro

A métrica F1 Score foi de: 0.8591866574453395
Tempo de processamento: 00:06:24.77


In [ ]:
#Tentativa 7 (Modelo RandomForestClassifier + Bigrama sem stopwords spacy)
start = time.time()

#Instanciando o modelo
vect = CountVectorizer(ngram_range=(2,2), stop_words=stops_spacy)
vect.fit(dados.text_pt)
text_vect = vect.transform(dados.text_pt)

X_train, X_test, y_train, y_test = train_test_split(
      text_vect, 
      dados["sentiment"], 
      test_size = 0.2, 
      random_state = 42
  )

modelo = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state = 42)

modelo.fit(X_train, y_train)

y_prediction = modelo.predict(X_test)

f1 = f1_score(y_prediction, y_test, average = 'weighted')
print("A métrica F1 Score foi de:", f1) 

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Tempo de processamento: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds)) 
## A métrica F1 Score foi de: 0.7926826080983602 com um tempo de processamento de 01:15:53.24
# Neste teste usamos bigrama com o RandomForestClassifier mas tivemos uma queda na métrica e um aumento muito grande no tempo de processamento

A métrica F1 Score foi de: 0.7926826080983602
Tempo de processamento: 01:15:53.24


In [ ]:
# Função para realizar a lematização completa do documento
def lemmatizer_text(text):        
  sent = []
  doc = nlp(text)
  for word in doc:
      sent.append(word.lemma_)
  return " ".join(sent)

# Função para realiazar a lematização para os verbos do documento
def lemmatizer_verbs(text):        
  sent = []
  doc = nlp(text)
  for word in doc:
      if word.pos_ == "VERB":
          sent.append(word.lemma_)
      else:
          sent.append(word.orth_)
  return " ".join(sent)

In [12]:
# Aplicando a funcao para lematizar o texto e inserir no dataframe
start = time.time()
dados['text_lemma'] = dados.text_pt.apply(lemmatizer_text)
end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Tempo de processamento: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds)) #00:52:39.19

Tempo de processamento: 00:50:01.98


In [13]:
# Aplicando a funcao para lematizar os verbos do texto e inserir no dataframe
start = time.time()
dados['text_lemma_verbs'] = dados.text_pt.apply(lemmatizer_verbs)
end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Tempo de processamento: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds)) #00:52:03.43

Tempo de processamento: 00:51:30.36


In [14]:
#Tentativa 8 (Modelo RandomForestClassifier + Unigrama sem stopwords spacy + lematização do texto)
start = time.time()

#Instanciando o modelo
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops_spacy)
vect.fit(dados.text_lemma)
text_vect = vect.transform(dados.text_lemma)

X_train, X_test, y_train, y_test = train_test_split(
      text_vect, 
      dados["sentiment"], 
      test_size = 0.2, 
      random_state = 42
  )

modelo = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state = 42)

modelo.fit(X_train, y_train)

y_prediction = modelo.predict(X_test)

f1 = f1_score(y_prediction, y_test, average = 'weighted')
print("A métrica F1 Score foi de:", f1) 

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Tempo de processamento: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds)) 
## A métrica F1 Score foi de: 0.8481625137610169 com um tempo de processamento de 00:02:57.31
# Com o texto lematizado não conseguimos aumentar o score.

A métrica F1 Score foi de: 0.8481625137610169
Tempo de processamento: 00:02:57.31


In [15]:
#Tentativa 8 (Modelo RandomForestClassifier + Bigrama sem stopwords spacy + lematização do texto)
start = time.time()

#Instanciando o modelo
vect = CountVectorizer(ngram_range=(2,2), stop_words=stops_spacy)
vect.fit(dados.text_lemma)
text_vect = vect.transform(dados.text_lemma)

X_train, X_test, y_train, y_test = train_test_split(
      text_vect, 
      dados["sentiment"], 
      test_size = 0.2, 
      random_state = 42
  )

modelo = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state = 42)

modelo.fit(X_train, y_train)

y_prediction = modelo.predict(X_test)

f1 = f1_score(y_prediction, y_test, average = 'weighted')
print("A métrica F1 Score foi de:", f1) 

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Tempo de processamento: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds)) 
## A métrica F1 Score foi de: 0.7976251138838961 com um tempo de processamento de 00:47:42.00
# Com o texto lematizado e bigramas tivemos uma piora no score.

A métrica F1 Score foi de: 0.7976251138838961
Tempo de processamento: 00:47:42.00


In [16]:
#Tentativa 9 (Modelo RandomForestClassifier + Unigrama sem stopwords spacy + lematização dos verbos)
start = time.time()

#Instanciando o modelo
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops_spacy)
vect.fit(dados.text_lemma_verbs)
text_vect = vect.transform(dados.text_lemma_verbs)

X_train, X_test, y_train, y_test = train_test_split(
      text_vect, 
      dados["sentiment"], 
      test_size = 0.2, 
      random_state = 42
  )

modelo = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state = 42)

modelo.fit(X_train, y_train)

y_prediction = modelo.predict(X_test)

f1 = f1_score(y_prediction, y_test, average = 'weighted')
print("A métrica F1 Score foi de:", f1) 

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Tempo de processamento: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds)) 
## A métrica F1 Score foi de: 0.8516232557984069 com um tempo de processamento de 00:03:04.04
# Com a lematização dos verbos não conseguimos aumentar o score.

A métrica F1 Score foi de: 0.8516232557984069
Tempo de processamento: 00:03:04.04


In [17]:
#Tentativa 10 (Modelo RandomForestClassifier + Bigrama sem stopwords spacy + lematização dos verbos)
start = time.time()

#Instanciando o modelo
vect = CountVectorizer(ngram_range=(2,2), stop_words=stops_spacy)
vect.fit(dados.text_lemma_verbs)
text_vect = vect.transform(dados.text_lemma_verbs)

X_train, X_test, y_train, y_test = train_test_split(
      text_vect, 
      dados["sentiment"], 
      test_size = 0.2, 
      random_state = 42
  )

modelo = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state = 42)

modelo.fit(X_train, y_train)

y_prediction = modelo.predict(X_test)

f1 = f1_score(y_prediction, y_test, average = 'weighted')
print("A métrica F1 Score foi de:", f1) 

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Tempo de processamento: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds)) 
## A métrica F1 Score foi de: 0.7938966605465549 com um tempo de processamento de 01:05:15.47
# Com a lematização dos verbos e bigramas tivemos uma queda no score.

A métrica F1 Score foi de: 0.7938966605465549
Tempo de processamento: 01:05:15.47


In [18]:
#Tentativa 11 (Modelo LogisticRegression + Bigrama sem stopwords spacy + lematização do texto)
start = time.time()

vect = CountVectorizer(ngram_range=(2,2), stop_words=stops_spacy)
vect.fit(dados.text_lemma)
text_vect = vect.transform(dados.text_lemma)

X_train, X_test, y_train, y_test = train_test_split(
      text_vect, 
      dados["sentiment"], 
      test_size = 0.2, 
      random_state = 42
  )

modelo = LogisticRegression()
modelo.fit(X_train, y_train)

y_prediction = modelo.predict(X_test)

f1 = f1_score(y_prediction, y_test, average = 'weighted')
print("A métrica F1 Score foi de:", f1) 

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Tempo de processamento: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
## A métrica F1 Score foi de: 0.8484572489046945 com um tempo de processamento de 00:01:22.29
# Usando lematização do texto e bigramas no modelo LogisticRegression não aumentamos o score

A métrica F1 Score foi de: 0.8484572489046945
Tempo de processamento: 00:01:22.29


In [19]:
#Tentativa 12 (Modelo LogisticRegression + Bigrama sem stopwords spacy + lematização do verbo)
start = time.time()

vect = CountVectorizer(ngram_range=(2,2), stop_words=stops_spacy)
vect.fit(dados.text_lemma_verbs)
text_vect = vect.transform(dados.text_lemma_verbs)

X_train, X_test, y_train, y_test = train_test_split(
      text_vect, 
      dados["sentiment"], 
      test_size = 0.2, 
      random_state = 42
  )

modelo = LogisticRegression()
modelo.fit(X_train, y_train)

y_prediction = modelo.predict(X_test)

f1 = f1_score(y_prediction, y_test, average = 'weighted')
print("A métrica F1 Score foi de:", f1) 

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Tempo de processamento: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
## A métrica F1 Score foi de: 0.8481918426296658 com um tempo de processamento de 00:01:15.01
# Usando lematização dos verbos e bigramas no modelo LogisticRegression não aumentamos o score

A métrica F1 Score foi de: 0.8481918426296658
Tempo de processamento: 00:01:15.01


In [30]:
#Tentativa 13 (Modelo DecisionTreeClassifier + Unigrama sem stopwords spacy + lematização do verbo)

start = time.time()

vect = CountVectorizer(ngram_range=(1,1), stop_words=stops_spacy)
vect.fit(dados.text_lemma_verbs)
text_vect = vect.transform(dados.text_lemma_verbs)

X_train, X_test, y_train, y_test = train_test_split(
      text_vect, 
      dados["sentiment"], 
      test_size = 0.2, 
      random_state = 42
  )

modelo = DecisionTreeClassifier(random_state=42)
modelo.fit(X_train, y_train)

y_prediction = modelo.predict(X_test)

f1 = f1_score(y_prediction, y_test, average = 'weighted')

print("A métrica F1 Score foi de:", f1)

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Tempo de processamento: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

## A métrica F1 Score foi de: 0.7210636911923664 com um tempo de processamento de 00:01:39.65
# Usando lematização dos verbos e unigrama no modelo DecisionTreeClassifier tivemos uma grande queda no score

A métrica F1 Score foi de: 0.7210636911923664
Tempo de processamento: 00:01:39.65


In [31]:
#Tentativa 14 (Modelo DecisionTreeClassifier + Bigrama sem stopwords spacy + lematização do verbo)

start = time.time()

vect = CountVectorizer(ngram_range=(2,2), stop_words=stops_spacy)
vect.fit(dados.text_lemma_verbs)
text_vect = vect.transform(dados.text_lemma_verbs)

X_train, X_test, y_train, y_test = train_test_split(
      text_vect, 
      dados["sentiment"], 
      test_size = 0.2, 
      random_state = 42
  )

modelo = DecisionTreeClassifier(random_state=42)
modelo.fit(X_train, y_train)

y_prediction = modelo.predict(X_test)

f1 = f1_score(y_prediction, y_test, average = 'weighted')

print("A métrica F1 Score foi de:", f1)

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Tempo de processamento: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

## A métrica F1 Score foi de: 0.7297999342312398 com um tempo de processamento de 00:25:02.04
# Usando lematização dos verbos e bigrama no modelo DecisionTreeClassifier tivemos uma grande queda no score

A métrica F1 Score foi de: 0.7297999342312398
Tempo de processamento: 00:25:02.04


## Resultados

### Conseguimos como melhor F1 Score 0.8591866574453395 do modelo "#Tentativa 6 (Modelo RandomForestClassifier + Unigrama sem stopwords spacy + n_estimators 200)"
### Também obtivemos bons números utilizando o modelo "#Tentativa 11 (Modelo LogisticRegression + Bigrama sem stopwords spacy + lematização do texto)" com um Score 0.8484572489046945